In [1]:
%load_ext autoreload
%autoreload 
!pip install pandas==1.5.3 openpyxl


import numpy as np
import seaborn as sns
import scipy
import skimage

sns.set_style("ticks")
from pathlib import Path

import pandas as pd
import pigleg_evaluation_tools as pet

# Get the data 
fn = sorted(list(Path(".").glob("*list_of_all_stitches*.xlsx")))[-1]
fn = sorted(list(Path(".").glob("*all_stitches_with_human_annotations*.xlsx")))[-1]
print(fn)
dfall = pd.read_excel(fn)

#remove unnamed columns
dfall = dfall.loc[:, ~dfall.columns.str.contains('^Unnamed')]

dfall


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: torch-tensorrt 1.1.0a0 has a non-standard dependency specifier torch>=1.10.0+cu113<1.11.0. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torch-tensorrt or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
all_stitches_with_human_annotations.2024-09-18_18-59-12.xlsx


,filename,annotation_annotation_annotation,annotation_annotation_time_and_movements,report_url,Stitch duration [s],Stitch start at [s],Needle holder stitch area presence [%],Needle holder stitch median area presence [%],Needle holder stitch length [m],Needle holder stitch visibility [s],...,Needle holder bbox piercing velocity std,Needle holder bbox piercing velocity above threshold,Needle holder bbox piercing velocity threshold [m/s],Needle holder bbox piercing unit,Needle holder bbox piercing visibility [%],done_by_expert,file_stitch,Carina,Ana,Mira
0,Ekn_Ana_3.mp4,00:00:01 stitch_start\n00:00:24 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,53.0,1.0,44.822485,89.053254,2.146999,45.066667,...,NaN,NaN,NaN,NaN,NaN,1,Ekn_Ana_3.mp4_0,4.2,4.333333,5.0
1,Ekn_Ana_2.mp4,00:00:02 stitch_start\n00:00:21 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,50.0,2.0,54.172989,94.992413,1.920482,43.933333,...,NaN,NaN,NaN,NaN,NaN,1,Ekn_Ana_2.mp4_0,4.6,4.666667,5.0
2,Ekn_Ana_4.mp4,00:00:01 stitch_start\n00:00:21 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,51.0,1.0,61.417323,94.015748,1.920755,42.333333,...,NaN,NaN,NaN,NaN,NaN,1,Ekn_Ana_4.mp4_0,3.2,3.666667,4.5
3,Ekn_Ana_1.mp4,00:00:01 stitch_start\n00:00:22 knot_end\n00:0...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,48.0,1.0,44.511278,68.270677,3.370340,44.333333,...,NaN,NaN,NaN,NaN,NaN,1,Ekn_Ana_1.mp4_0,4.4,4.333333,5.0
4,Ekn_Ana_5.mp4,00:00:00 stitch_start\n00:00:17 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,43.0,0.0,36.909091,84.000000,2.854883,36.666667,...,NaN,NaN,NaN,NaN,NaN,1,Ekn_Ana_5.mp4_0,3.2,3.666667,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,Schiesser_David_post_test.mp4,00:00:04 stitch_start\n00:00:47 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,58.0,195.0,78.386167,93.227666,2.580827,46.266667,...,NaN,NaN,NaN,NaN,NaN,0,Schiesser_David_post_test.mp4_3,3.8,4.250000,3.0
89,Einzelknopfnaht_Schwein_Kerstin_Preuß.mp4,00:00:06 stitch_start\n00:00:32 knot_start \n0...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,50.0,240.0,83.281250,93.125000,2.302644,42.666667,...,NaN,NaN,NaN,NaN,NaN,0,Einzelknopfnaht_Schwein_Kerstin_Preuß.mp4_4,3.8,3.500000,4.0
90,Stresstest_Einzelknopfnaht.mp4,Das Ergebnis finde ich sehr schön. Die Wundrän...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,54.0,348.0,89.949749,99.874372,1.301011,53.066667,...,NaN,NaN,NaN,NaN,NaN,0,Stresstest_Einzelknopfnaht.mp4_4,3.6,3.750000,5.0
91,Bildschirmaufnahme_2022-10-11_um_11.20.24.mov,00:00:17 stitch_start\n00:01:41 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,101.0,698.0,72.877536,91.885800,2.801612,88.733333,...,NaN,NaN,NaN,NaN,NaN,0,Bildschirmaufnahme_2022-10-11_um_11.20.24.mov_4,3.2,4.000000,4.0


In [8]:
annotators = ["Carina", "Ana", "Mira"]

dfall["mean_movement_annotation"] = dfall[annotators].mean(axis=1)
dfall.mean_movement_annotation.describe()

count    93.000000
mean      3.610394
std       0.805056
min       1.355556
25%       3.183333
50%       3.733333
75%       4.150000
max       5.000000
Name: mean_movement_annotation, dtype: float64

In [6]:
# pick numeric columns
numeric_columns = dfall.select_dtypes(include=[np.number]).columns


# Remove NaN values

In [17]:
# kick rows with a lot missing values
missing_values_rows = dfall.isnull().sum(axis=1)
missing_values_rows.sort_values(ascending=False)
kicked_lines = dfall[missing_values_rows > 100]
dfall = dfall.drop(kicked_lines.index)
kicked_lines

,filename,annotation_annotation_annotation,annotation_annotation_time_and_movements,report_url,Stitch duration [s],Stitch start at [s],Needle holder stitch area presence [%],Needle holder stitch median area presence [%],Needle holder stitch length [m],Needle holder stitch visibility [s],...,Needle holder bbox piercing velocity above threshold,Needle holder bbox piercing velocity threshold [m/s],Needle holder bbox piercing unit,Needle holder bbox piercing visibility [%],done_by_expert,file_stitch,Carina,Ana,Mira,mean_movement_annotation
25,Ekn_CSW_3.mp4,NaN,NaN,http://suran.kky.zcu.cz:8000//uploader/web_rep...,52.666667,5.133333,0.000000,53.437500,1.845188,21.333333,...,NaN,NaN,NaN,NaN,1,Ekn_CSW_3.mp4_0,3.2,3.333333,5.0,3.844444
74,"Hilger,Anke-Posttest.mp4",00:00:06 stitch_start\n00:00:45 knot\n00:01:02...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,43.000000,82.000000,64.053537,88.527725,1.787032,34.866667,...,NaN,NaN,NaN,NaN,0,"Hilger,Anke-Posttest.mp4_1",4.2,4.000000,4.0,4.066667
77,Einzelknopfnaht_Elena.Vai-Wieting.mov,00:00:02 stitch_start\n00:00:43 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,92.000000,216.000000,50.640478,94.022203,2.709030,78.066667,...,NaN,NaN,NaN,NaN,0,Einzelknopfnaht_Elena.Vai-Wieting.mov_2,3.4,3.333333,5.0,3.911111
82,"Hilger,Anke-Posttest.mp4",00:00:06 stitch_start\n00:00:45 knot\n00:01:02...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,56.000000,138.000000,62.274619,93.897365,1.870209,48.066667,...,NaN,NaN,NaN,NaN,0,"Hilger,Anke-Posttest.mp4_2",3.8,3.333333,3.0,3.377778
87,"Hilger,Anke-Posttest.mp4",00:00:06 stitch_start\n00:00:45 knot\n00:01:02...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,41.000000,205.000000,70.746888,90.248963,1.742624,32.133333,...,NaN,NaN,NaN,NaN,0,"Hilger,Anke-Posttest.mp4_3",3.8,3.666667,4.0,3.822222
92,"Hilger,Anke-Posttest.mp4",00:00:06 stitch_start\n00:00:45 knot\n00:01:02...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,43.000000,258.000000,73.928571,92.321429,1.518294,37.333333,...,NaN,NaN,NaN,NaN,0,"Hilger,Anke-Posttest.mp4_4",3.8,3.666667,4.0,3.822222


In [20]:
# from numeric columns remove columns with lot of missing values
missing_values_columns = dfall[numeric_columns].isnull().sum()
missing_values_columns.sort_values(ascending=False)
kicked_columns = missing_values_columns[missing_values_columns > 20]
dfall = dfall.drop(kicked_columns.index, axis=1)

In [55]:
relevant_column = "mean_movement_annotation"
annotation_columns = annotators + ["mean_movement_annotation", "movement evaluation stitch", "done_by_expert", "annotation_annotation_time_and_movements"]
id_numeric_columns = ["stitch_id"]
sample_id_cols = ["filename", "stitch_id"]
predicted_columns = [relevant_column]  # list(corr.columns[corr.columns.str.contains("is_pretest_num")])
predicted_columns_extended = [relevant_column]  + annotators

numeric_columns = dfall.select_dtypes(include=[np.number]).columns
measured_columns = numeric_columns.drop(annotation_columns + id_numeric_columns)
dfall.shape

(87, 117)

In [34]:
# measured_columns

Index(['Stitch duration [s]', 'Stitch start at [s]',
       'Needle holder stitch area presence [%]',
       'Needle holder stitch median area presence [%]',
       'Needle holder stitch length [m]',
       'Needle holder stitch visibility [s]', 'Needle holder stitch velocity',
       'Needle holder stitch velocity std',
       'Needle holder stitch velocity above threshold',
       'Needle holder stitch velocity threshold [m/s]',
       'Needle holder stitch visibility [%]',
       'Forceps stitch area presence [%]',
       'Forceps stitch median area presence [%]', 'Forceps stitch length [m]',
       'Forceps stitch visibility [s]', 'Forceps stitch velocity',
       'Forceps stitch velocity std',
       'Forceps stitch velocity above threshold',
       'Forceps stitch velocity threshold [m/s]',
       'Forceps stitch visibility [%]', 'Scissors stitch area presence [%]',
       'Scissors stitch median area presence [%]',
       'Scissors stitch length [m]', 'Scissors stitch visibility

In [39]:
# measured_columns.append("mean_movement_annotation")

TypeError: all inputs must be Index

In [40]:
# correlation of all columns with the mean_movement_annotation
# correlations

mean_movement_annotation                           1.000000
Stitch duration [s]                                0.798597
Piercing duration [s]                              0.782844
Knot duration [s]                                  0.711262
Needle holder piercing visibility [s]              0.667457
                                                     ...   
Needle holder knot velocity threshold [m/s]             NaN
Forceps knot velocity threshold [m/s]                   NaN
Scissors knot velocity threshold [m/s]                  NaN
Needle holder piercing velocity threshold [m/s]         NaN
Forceps piercing velocity threshold [m/s]               NaN
Name: mean_movement_annotation, Length: 91, dtype: float64

In [146]:
dfst = dfall
corr = dfall[list(measured_columns) + [relevant_column]].corr()["mean_movement_annotation"].abs().sort_values(ascending=False).head(21)
# corr = dfst.corr(numeric_only=True)[relevant_column].abs().sort_values(ascending=False).head(10)
corr = corr[corr.index != relevant_column]
cols_X = list(corr.index)
corr


Stitch duration [s]                                      0.798597
Piercing duration [s]                                    0.782844
Knot duration [s]                                        0.711262
Needle holder piercing visibility [s]                    0.667457
Needle holder stitch visibility [s]                      0.661771
Needle holder to forceps stitch below threshold [s]      0.660344
Needle holder to forceps piercing below threshold [s]    0.647396
Forceps stitch visibility [s]                            0.606019
Forceps piercing visibility [s]                          0.598836
Needle holder stitch velocity above threshold            0.561988
Needle holder knot visibility [s]                        0.521281
Scissors stitch visibility [s]                           0.497707
Forceps stitch length [m]                                0.492124
Forceps piercing length [m]                              0.489134
Needle holder piercing length [m]                        0.480851
Needle hol

# Train a predictor

In [96]:

import sklearn
# train a model to predict stars_auto
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error, r2_score
# import SVM
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
# sample_id_cols = ["filename", "stitch_id"]
#train a model to predict stars_auto


In [153]:
cols_selection = "corr"
cols_selection = "manual"

if cols_selection == "manual":
    cols_X = [
        'Stitch duration [s]',
     # 'Piercing duration [s]',
     # 'Knot duration [s]',
     # 'Needle holder piercing visibility [s]',
     # 'Needle holder piercing visibility [s]',
     'Needle holder stitch visibility [s]',
     'Needle holder to forceps stitch below threshold [s]',
     # 'Needle holder to forceps piercing below threshold [s]',
     'Needle holder stitch velocity above threshold',
     'Forceps stitch visibility [s]',
     # 'Forceps piercing visibility [s]',
    'Scissors stitch visibility [s]',
        "Forceps stitch length [m]",
    ]




In [154]:

dfsc = dfst[
    dfst["Stitch duration [s]"].notna() &
    dfst["Needle holder stitch visibility [s]"].notna() &
    dfst[predicted_columns[0]].notna()
    # dfs[predicted_columns].notna()
    ]
display(dfsc)
cols_X_with_sample_id = sample_id_cols + cols_X

dfsc[cols_X_with_sample_id].isna().sum().sort_values(ascending=False)

display(dfsc[cols_X_with_sample_id].isna().sum().sort_values(ascending=False))

# fill 0 instead of None
# dfsc = dfsc[sample_id_cols + predicted_columns + cols_X].copy()
dfsc = dfsc[sample_id_cols + predicted_columns_extended + cols_X].copy()
dfsc[cols_X] = dfsc[cols_X].fillna(0)

print(len(dfsc))


,filename,annotation_annotation_annotation,annotation_annotation_time_and_movements,report_url,Stitch duration [s],Stitch start at [s],Needle holder stitch area presence [%],Needle holder stitch median area presence [%],Needle holder stitch length [m],Needle holder stitch visibility [s],...,Needle holder to forceps piercing below threshold [%],Forceps bbox piercing unit,stitch_id,Needle holder bbox piercing unit,done_by_expert,file_stitch,Carina,Ana,Mira,mean_movement_annotation
0,Ekn_Ana_3.mp4,00:00:01 stitch_start\n00:00:24 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,53.0,1.0,44.822485,89.053254,2.146999,45.066667,...,77.681159,NaN,0,NaN,1,Ekn_Ana_3.mp4_0,4.2,4.333333,5.0,4.511111
1,Ekn_Ana_2.mp4,00:00:02 stitch_start\n00:00:21 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,50.0,2.0,54.172989,94.992413,1.920482,43.933333,...,91.929825,NaN,0,NaN,1,Ekn_Ana_2.mp4_0,4.6,4.666667,5.0,4.755556
2,Ekn_Ana_4.mp4,00:00:01 stitch_start\n00:00:21 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,51.0,1.0,61.417323,94.015748,1.920755,42.333333,...,81.000000,NaN,0,NaN,1,Ekn_Ana_4.mp4_0,3.2,3.666667,4.5,3.788889
3,Ekn_Ana_1.mp4,00:00:01 stitch_start\n00:00:22 knot_end\n00:0...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,48.0,1.0,44.511278,68.270677,3.370340,44.333333,...,61.111111,NaN,0,NaN,1,Ekn_Ana_1.mp4_0,4.4,4.333333,5.0,4.577778
4,Ekn_Ana_5.mp4,00:00:00 stitch_start\n00:00:17 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,43.0,0.0,36.909091,84.000000,2.854883,36.666667,...,81.960784,NaN,0,NaN,1,Ekn_Ana_5.mp4_0,3.2,3.666667,5.0,3.955556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,Bildschirmaufnahme_2022-10-11_um_11.20.24.mov,00:00:17 stitch_start\n00:01:41 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,121.0,552.0,82.840237,92.504931,3.348492,101.400000,...,70.752688,NaN,3,NaN,0,Bildschirmaufnahme_2022-10-11_um_11.20.24.mov_3,2.6,3.000000,4.0,3.200000
88,Schiesser_David_post_test.mp4,00:00:04 stitch_start\n00:00:47 knot_start\n00...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,58.0,195.0,78.386167,93.227666,2.580827,46.266667,...,89.473684,NaN,3,NaN,0,Schiesser_David_post_test.mp4_3,3.8,4.250000,3.0,3.683333
89,Einzelknopfnaht_Schwein_Kerstin_Preuß.mp4,00:00:06 stitch_start\n00:00:32 knot_start \n0...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,50.0,240.0,83.281250,93.125000,2.302644,42.666667,...,89.487179,NaN,4,NaN,0,Einzelknopfnaht_Schwein_Kerstin_Preuß.mp4_4,3.8,3.500000,4.0,3.766667
90,Stresstest_Einzelknopfnaht.mp4,Das Ergebnis finde ich sehr schön. Die Wundrän...,1.0,http://suran.kky.zcu.cz:8000//uploader/web_rep...,54.0,348.0,89.949749,99.874372,1.301011,53.066667,...,75.072464,NaN,4,NaN,0,Stresstest_Einzelknopfnaht.mp4_4,3.6,3.750000,5.0,4.116667


Scissors stitch visibility [s]                         15
filename                                                0
stitch_id                                               0
Stitch duration [s]                                     0
Needle holder stitch visibility [s]                     0
Needle holder to forceps stitch below threshold [s]     0
Needle holder stitch velocity above threshold           0
Forceps stitch visibility [s]                           0
Forceps stitch length [m]                               0
dtype: int64

87


In [155]:
cols_X

['Stitch duration [s]',
 'Needle holder stitch visibility [s]',
 'Needle holder to forceps stitch below threshold [s]',
 'Needle holder stitch velocity above threshold',
 'Forceps stitch visibility [s]',
 'Scissors stitch visibility [s]',
 'Forceps stitch length [m]']

In [148]:
# from sklearn.model_selection import train_test_split
# # split dataset into train and test
# 
# X = dfall[id_columns + list(measured_columns)] # .drop(annotation_columns, axis=1)
# y = dfall["mean_movement_annotation"]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 
# X_train_id = X_train[id_columns]
# X_test_id = X_test[id_columns]
# 
# X_train = X_train.drop(id_columns, axis=1)
# X_test = X_test.drop(id_columns, axis=1)

In [156]:
# split train and test 
dfsc.reset_index(inplace=True)
train_idxs, test_idxs = train_test_split(dfsc.index, test_size=0.2, random_state=42)

In [157]:
eval_results = []

# Train it for different number of columns with mean of annotators

In [158]:
def n_params_mplregresor(model):
    """Return total number of parameters in a 
    Scikit-Learn model.

    This works for the following model types:
     - sklearn.neural_network.MLPClassifier
     - sklearn.neural_network.MLPRegressor
     - sklearn.linear_model.LinearRegression
     - and maybe some others
    """
    return (sum([a.size for a in model.coefs_]) +  
            sum([a.size for a in model.intercepts_]))

In [216]:
maximum_number_of_columns = 5
maximum_number_of_columns = len(cols_X)

dataset_type  = "mean"
# dataset_type  = "all"

results = {}

models = []
best_clf = None
best_score = -1
best_cols = None
for i in range(1, maximum_number_of_columns + 1):
    for predicted_column in predicted_columns:
        # if True:
        #     predicted_column = "annotation_annotation_stars"
        data_cols = cols_X[:i]
        first_columns = data_cols + sample_id_cols
        cols = first_columns.copy()
        # cols.append(predicted_column)
        cols.extend(predicted_columns_extended)


        dfsx = dfsc[cols].copy()
        # dfsx = dfst[cols].copy()
        l1 = len(dfsx)
        dfsx = dfsx.dropna()
        l2 = len(dfsx)
        assert l1 == l2, "There are still NA values in the data"
        X = dfsx[first_columns].drop(sample_id_cols, axis=1)
        # print(f"{X.columns=}")
        # X = X.drop(predicted_column, axis=1)
        if dataset_type == "mean":
            y = dfsx[predicted_column]
            X_train = X.loc[train_idxs]
            X_test = X.loc[test_idxs]
            y_train = y.loc[train_idxs]
            y_test = y.loc[test_idxs]
        elif dataset_type == "all":
            list_y = []
            list_X_train = []
            list_X_test = []
            list_y_train = []
            list_y_test = []
            for a in annotators:
                y = dfsx[a]
                list_y.append(dfsx[a])
                list_X_train.append(X.loc[train_idxs])
                list_X_test.append(X.loc[test_idxs])
                list_y_train.append(y.loc[train_idxs])
                list_y_test.append(y.loc[test_idxs])
            
            y = pd.concat(list_y, axis=0)
            X_train = pd.concat(list_X_train, axis=0)
            X_test = pd.concat(list_X_test, axis=0)
            y_train = pd.concat(list_y_train, axis=0)
            y_test = pd.concat(list_y_test, axis=0)
            # print(f"{y.shape=} {X.shape=} {X_train.shape=} {X_test.shape=} {y_train.shape=} {y_test.shape=}")
            
                  
            
            
                
        # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        n_pars = 0
        # train regresion
        
        # clf = LinearRegression()
        # clf = SVR()
        
        # clf = MLPRegressor(random_state=1, max_iter=10000, hidden_layer_sizes=(7,4,), activation="relu", solver="adam")
        clf = MLPRegressor(random_state=1, max_iter=10000, hidden_layer_sizes=(5,5,5), activation="relu", solver="adam") # score=0.82
        
        # score
        # The coefficient R^2 is defined as (1 - u/v), 
        # where u is the regression sum of squares ((y_true - y_pred) ** 2).sum() 
        # and v is the residual sum of squares ((y_true - y_true.mean()) ** 2).sum().
        # Best possible score is 1.0, lower values are worse.
        # print(X_train.shape, len(X_train), len(X_test), predicted_column)
        clf.fit(X_train, y_train)
        if clf.__class__.__name__ == "MLPRegressor":
            n_pars = n_params_mplregresor(clf)
        elif clf.__class__.__name__ == "LinearRegression":
            n_pars = len(clf.coef_)
        elif clf.__class__.__name__ == "SVR":
            n_pars = len(clf.support_vectors_)
            
        clf_score = clf.score(X_test, y_test)
        pred_test = clf.predict(X_test)
        pred_train = clf.predict(X_train)
        results[predicted_column] = {"score": clf.score(X_test, y_test), "model": clf}
        avg_error = np.mean(np.abs(pred_test - y_test))
        print(f"{X_train.shape} {len(X_train)} {len(X_test)} {predicted_column} {n_pars} {avg_error} {clf_score}")
        clf_name = clf.__class__.__name__
        # save model
        model_save_dict=dict(
            model_name=clf_name,
            model=clf,
            data_cols=data_cols,
            sample_id_cols=sample_id_cols,
            predicted_columns=predicted_columns,
        )
        import pickle
        fn = f"model_{clf_name}_{dataset_type}_{predicted_column}_{i}.pkl"
        with open(fn, "wb") as f:
            pickle.dump(model_save_dict, f)
        # print(f"Model saved to {fn}")
        if best_clf is None or best_score < clf_score:
            best_clf = model_save_dict
            best_score = clf_score
        models.append(clf)
        eval_results.append(dict(
            timestamp=pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S"),
            model_name=clf_name,
            score=clf_score,
            n_columns=i,
            filename=fn,
            dataset_type=dataset_type,
            train_size=len(X_train),
            test_size=len(X_test),
            avg_error=avg_error,
            n_params = n_pars,
            cols_selection=cols_selection,
        ))
fn_best = f"model_best_{best_clf['model_name']}.pkl"
with open(fn_best, "wb") as f:
    pickle.dump(best_clf, f)
print(f"{best_clf['model_name']}")



(69, 1) 69 18 mean_movement_annotation 76 3.0630152020528705 -20.8908557213005
(69, 2) 69 18 mean_movement_annotation 81 0.6789926356292912 -0.21015952313207764
(69, 3) 69 18 mean_movement_annotation 86 0.2785065769326065 0.7794561048000892
(69, 4) 69 18 mean_movement_annotation 91 0.2506165658073139 0.8239018105899056
(69, 5) 69 18 mean_movement_annotation 96 0.6250027119334326 -0.9817269342180606
(69, 6) 69 18 mean_movement_annotation 101 0.6497679554051771 -0.8010291038963848
(69, 7) 69 18 mean_movement_annotation 106 0.517804203366607 -0.013920143705199228
MLPRegressor


In [170]:


X = dfsc[best_clf["data_cols"]]
X_test = X.loc[test_idxs]
y_test = dfsc[predicted_column].loc[test_idxs]

df_test = pd.DataFrame(dict(
    prediction=best_clf["model"].predict(X_test),
    true=y_test
))

df_test["error"] = df_test["prediction"] - df_test["true"]
df_test["error_abs"] = df_test["error"].abs()
display(df_test["error_abs"].describe())
df_test



count    18.000000
mean      0.250617
std       0.179677
min       0.017631
25%       0.150835
50%       0.195534
75%       0.280008
max       0.580276
Name: error_abs, dtype: float64

,prediction,true,error,error_abs
76,3.966958,4.133333,-0.166375,0.166375
0,4.008933,4.511111,-0.502179,0.502179
26,3.762152,3.733333,0.028819,0.028819
22,3.809901,3.955556,-0.145655,0.145655
12,4.148808,4.333333,-0.184526,0.184526
67,3.009149,3.266667,-0.257518,0.257518
10,4.148808,4.222222,-0.073414,0.073414
18,4.170296,4.266667,-0.096371,0.096371
4,4.146621,3.955556,0.191066,0.191066
68,3.902498,3.322222,0.580276,0.580276


In [ ]:
# Save information about train/test split

In [195]:
dfsc["is_train"] = np.NaN
dfsc.loc[train_idxs, "is_train"] = 1.
dfsc.loc[test_idxs, "is_train"] = 0.

dfst["is_train"] = dfsc["is_train"]
dfst.value_counts("is_train")


is_train
1.0    65
0.0    18
dtype: int64

# Use predictor to predict all values



In [217]:
!ls -l model_best_*.pkl

-rw-r--r-- 1 appuser sudo   779 Sep 19 00:22 model_best_LinearRegression.pkl
-rw-r--r-- 1 appuser sudo 20788 Sep 19 00:22 model_best_MLPRegressor.pkl
-rw-r--r-- 1 appuser sudo  4091 Sep 19 00:22 model_best_SVR.pkl


In [225]:
predictors = ["model_best_LinearRegression.pkl", "model_best_MLPRegressor.pkl", "model_best_SVR.pkl"]



for predictor in predictors:
    with open(predictor, "rb") as f:
        model = pickle.load(f)
    X = dfst[model["data_cols"]]
    y = model["model"].predict(X)
    new_col = f"{model['model_name']} {model['predicted_columns'][0]}"
    dfst[new_col] = y
    
    print(f"{model['predicted_columns'][0]} {model['model'].score(X, dfst[model['predicted_columns'][0]])} {model['model_name']} ")
    dfst[model["predicted_columns"]].describe()


mean_movement_annotation 0.7193070519191431 LinearRegression 
mean_movement_annotation 0.6889825926262045 MLPRegressor 
mean_movement_annotation 0.7418289901501056 SVR 


In [227]:
timestamp = pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")
dfst.to_excel(f"all_stitches_with_human_annotations_and_predictions.{timestamp}.xlsx")